# Iterative Phase Estimation via the Cloud

This sample code and notebook was written by members of KPMG Quantum team in Australia. It aims to demonstrate expanded capabilities of Basic Measurement Feedback targets and makes use of bounded loops, classical function calls at run time, nested conditional if statements, mid circuit measurements and qubit reuse. 


### Two Dimensional Inner Product Calculation Using Iterative Phase Estimation on Three Qubits

This notebook demonstrates an iterative phase estimation within Q#. It will use iterative phase estimation to calculate an inner product between two 2-dimensional vectors encoded on a target qubit and an ancilla qubit. An additional control qubit is also initialised which will be the only qubit used for measurement.

The circuit begins by encoding the pair of vectors on the target qubit and the ancilla qubit. It then applies an Oracle operator to the entire register, controlled off the control qubit (which is set up in the $\ket +$ state). The controlled Oracle operator generates a phase on the $\ket 1$ state of the control qubit. This can then be read by applying a H gate to the control qubit to make the phase observable when measuring.



In [ ]:
%azure.connect ""

##### Open Name Spaces Required
There are a number of Name Spaces required for this set of code. They are opened in the block below.

In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

## Encoding vectors

The vectors v and c are to be encoded onto the target qubit and the ancilla qubit. The vector $v = (cos(\frac{\theta_1}{2}),sin(\frac{\theta_1}{2}))$ can be represented by the quantum state $\ket v = cos(\frac{\theta_1}{2})\ket 0 + sin(\frac{\theta_1}{2})\ket 1$, similarly $c$ can be constructed using $\theta_2$. 

A Y rotation applied to a target qubit in the $\ket 0$ state:

$$RY(\theta)\ket 0 = e^{iY\theta/2}\ket 0 = cos(\frac{\theta}{2})\ket 0 + sin(\frac{\theta}{2})\ket 1$$

**Note**: <u> A factor of 2 </u> is present here on theta. An application of a $RY(2\pi)$ gate on $\ket 0$ gives the state $-\ket 0$ and would encode the vector $(-1,0)$. This phase cannot be considered a global phase and removed as the entire register will be entangled.

The register of the target qubit and ancilla qubit is,

$$\ket  \Psi = \ket {\Psi_\text{Target qubit}}\ket {\Psi_\text{Ancilla qubit}}$$
The state to be created is on the target qubit and the ancilla qubit is,

$$\ket{\Psi}=\frac{1}{\sqrt{2}}(\ket{v}\ket{+}+\ket{c}\ket{-}),$$


which also takes the form,

$$\ket{\Psi} = \frac{1}{2}(\ket{v}+\ket{c})\ket{0}+\frac{1}{2}(\ket{v}-\ket{c})\ket{1}.$$


In [ ]:
operation StateInitialisation(TargetReg : Qubit, AncilReg : Qubit, theta_1 : Double, theta_2 : Double) : Unit is Adj + Ctl { //This is state preperation operator A for encoding the 2D vector (page 7)
    H(AncilReg);

    Controlled R([AncilReg], (PauliY, theta_1, TargetReg));        // Arbitrary controlled rotation based on theta. This is vector v.
                                                        
    X(AncilReg);                                                   // X gate on ancilla to change from |+> to |->.
    Controlled R([AncilReg], (PauliY, theta_2, TargetReg));        // Arbitrary controlled rotation based on theta. This is vector c.
    X(AncilReg);                                                  
    H(AncilReg);                                                  
}

## The Oracle

An oracle G needs to be constructed such that it generates an eigenphase on the state encoded on the target qubit and the ancilla qubit. The construction of this oracle is unimportant to the demonstration within this notebook, but the operation it applies is,

$$G\ket \Psi = e^{2\pi i\phi} \ket \Psi.$$

where the inner product $\braket {v|c}$ is contained within the phase $\phi$, which is bound between [0,1]. When applied controlled on the control qubit which begins in that state $\ket{\Psi_\text{Control Qubit}} = \ket +$,

$$\begin{aligned}
    \text{Controlled }G \ket{\Psi_\text{Control Qubit}} \ket \Psi  & = \frac {1}{\sqrt{2}} (\ket 0 \ket \Psi + e^{2\pi i\phi}\ket 1 \ket \Psi )\\
    & =\frac {1}{\sqrt{2}} (\ket 0 + e^{2\pi i\phi}\ket 1) \ket \Psi
\end{aligned}$$

Now the control qubit contains the phase $\phi$ which relates to the inner product $\braket {v|c}$

$$\ket{\Psi_\text{Control Qubit}} = \frac {1}{\sqrt{2}} (\ket 0 + e^{2\pi i\phi}\ket 1)$$

In [ ]:
operation GOracle(TargetReg : Qubit, AncilReg : Qubit, theta_1 : Double, theta_2 : Double) : Unit is Adj + Ctl {
    Z(AncilReg);
    within {
        Adjoint StateInitialisation(TargetReg, AncilReg, theta_1, theta_2);
        X(AncilReg);                                                        // Apply X gates individually here as currently ApplyAll is not Adj + Ctl
        X(TargetReg);
    }   
    apply {
        Controlled Z([AncilReg],TargetReg);
    }
}

## Iteration

Now for the iterative part of the circuit. For n measurements, consider that the phase can be represented as a binary value $\phi$, and that applying $2^n$ oracles makes the nth binary point of the phase observable (through simple binary multiplication, and modulus $2\pi$). The value of the control qubit can be readout, placed in a classical register and the qubit reset for use in the next iteration. The next iteration applies $2^{n-1}$ oracles, correcting phase on the control qubit dependent on the nth measurement. The state on the control qubit can be represented as,

$$ \ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i\phi}\ket 1 $$

where $\phi = 0.\phi_0\phi_1\phi_2\phi_3$...

Applying $2^n$ controlled oracles gives the state on the control qubit,

$$ G^{2^n}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\phi_n\phi_{n+1}\phi_{n+2}\phi_{n+3}...}\ket 1 $$

Consider that the phase has no terms deeper than $\phi_n$ (ie, terms $\phi_{n+1},\phi_{n+2}, \text{etc}$),

$$ G^{2^n}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\phi_n}\ket 1 $$

Now the value $\phi_n$ can be observed with a H gate and a measurement projecting along the Z axis. Resetting the control qubit and applying the oracle $2^{n-1}$ times,

$$ G^{2^{n-1}}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\phi_{n-1}\phi_n}\ket 1 $$

Using the previous measured value for $\phi_n$, the additional binary point can be rotated out.

$$ RZ(-2\pi \times 0.0\phi_n)G^{n-1}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\phi_{n-1}}\ket 1 $$

This process is iteratively applied for some bit precision n to obtain the phase $0.\phi_0\phi_1\phi_2...\phi_{n}$. The value is stored as a binary value $x = \phi_0\phi_1\phi_2...\phi_{n}$ as only integers are manipulatable at runtime currently.

As the readout tells nothing of either vector, only the inner product between them, the states on the target qubit and ancilla qubit <u>remain in the same state</u> throughout the process!

In [ ]:
operation IterativePhaseEstimation(TargetReg : Qubit, AncilReg : Qubit, theta_1 : Double, theta_2 : Double, Measurements : Int) : Int{
    use ControlReg = Qubit();
    mutable MeasureControlReg = [Zero, size = Measurements];
    mutable bitValue = 0;
    StateInitialisation(TargetReg, AncilReg, theta_1, theta_2);                                       //Apply to initialise state, this is defined by the angles theta_1 and theta_2
    for index in 0 .. Measurements - 1{                                                             
        H(ControlReg);                                                                              
        if index > 0 {                                                                              //Don't apply rotation on first set of oracles
            for index2 in 0 .. index - 1{                                                           //Loop through previous results
                if MeasureControlReg[Measurements - 1 - index2] == One{
                    let angle = -IntAsDouble(2^(index2))*PI()/(2.0^IntAsDouble(index));                   
                    R(PauliZ, angle, ControlReg);                                                  //Rotate control qubit dependent on previous measurements and number of measurements 
                }
            }
            
        }
        let powerIndex = (1 <<< (Measurements - 1 - index));
        for _ in 1 .. powerIndex{                                                                   //Apply a number of oracles equal to 2^index, where index is the number or measurements left
                Controlled GOracle([ControlReg],(TargetReg, AncilReg, theta_1, theta_2));
            }
        H(ControlReg);
        set MeasureControlReg w/= (Measurements - 1 - index) <- MResetZ(ControlReg);                //Make a measurement mid circuit 
        if MeasureControlReg[Measurements - 1 - index] == One{
            set bitValue += 2^(index);                                                              //Assign bitValue based on previous measurement
                                                                                                    
        }
    }
    return bitValue;
}

Finally to calculate the inner product from the measured value,

$$\braket {v|c} = -cos(2\pi x / 2^n)$$

where $x = \phi_0\phi_1\phi_2...\phi_{n}$. The denominator within the cosine function is to shift the binary point to match the original value $\phi$.

**Note**: For inner product that are not -1 or 1, the solutions are paired with a value difference of $2^{n-1}$. For example for n=3 measurements, the measured bit value of 2 would also have a pair solution of 6. Either of these values produce the same value of the inner product when input as the variable to the even function cosine (resulting in an inner product of 0 in this example).

**Note**: For inner product solutions between the discrete bit precision, a distribution of results will be produced based on where the inner product lies between the discrete bit value. 

In [ ]:
function CalculateInnerProduct(Results : Int, theta_1 : Double, theta_2 : Double, Measurements : Int): Unit{
    let DoubleVal = PI() * IntAsDouble(Results) / IntAsDouble(2 ^ (Measurements-1));
    let InnerProductValue = -Cos(DoubleVal);                                                      //Convert to the final inner product
    Message("The Bit Value measured is:");
    Message($"{Results}");
    Message("The Inner Product is:");
    Message($"{InnerProductValue}");
    Message("The True Inner Product is:");
    Message($"{Cos(theta_1/2.0)*Cos(theta_2/2.0)+Sin(theta_1/2.0)*Sin(theta_2/2.0)}");
}

In [ ]:
operation InnerProduct() : (Int, Double, Double, Int){
    let theta_1 = 0.0;                                                                           //Specify the angles for inner product
    let theta_2 = 0.0;
    let Measurements = 3;
    use TargetReg = Qubit();                                                                    //Create target register
    use AncilReg = Qubit();                                                                     //Create ancilla register
    let Results = IterativePhaseEstimation(TargetReg, AncilReg, theta_1, theta_2, Measurements);//This runs iterative phase estimation
    Reset(TargetReg);                                           
    Reset(AncilReg);
    return (Results, theta_1, theta_2, Measurements);
}

In [ ]:
operation SimulateInnerProduct() : Unit{                                                        //Operation for calculating the inner product on local simulators
        let (Results, theta_1, theta_2, Measurements) = InnerProduct();                             //This operation will output additional classical calculations
        CalculateInnerProduct(Results, theta_1, theta_2, Measurements);                             
    }

In [ ]:
%simulate SimulateInnerProduct

The inner product operation is reconstructed to fit within the requirements of the target. In this case, calls like "Message" and manipulation of doubles is not allowed. Therefore the output will be the bit value as generated by the IterativePhaseEstimation operation.

In [ ]:
operation HardwareInnerProduct() : Int{                                                         //Operation for calculating the inner product on hardware or emulators
    let (Results,_,_,_) = InnerProduct();
    return Results;                                                                             
}

Specify the target.

**Note**: The target requires a target execution profile that supports basic measurement feedback.

In [ ]:
%azure.target quantinuum.sim.h1-1e

In [ ]:
%azure.target-capability AdaptiveExecution

Submit the job to the target. The circuit is approximately 1.2 HQC each shot. The number of shots specified is 128, but this can be increased to reduce the variance of the result, up to some stable distribution.

**Note**: Choosing input parameters which only has one solution state (inner produces of -1 or 1) are ideal for visibility at a low number of shots.

In [ ]:
%azure.submit HardwareInnerProduct shots = 128


Check the status of the job specified.

In [ ]:
%azure.status

When the job is complete, output a histogram of results.

The results should show a majority in the solution states.

In [ ]:
%azure.output